# Random Forest
Port from Alien to Air.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-05-26 09:24:45.258955
Python 3.8.3
sklearn 1.1.1


[]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [3]:
# Data straight from CellProfiler (rows for every patch image file)
BASE_DIR='//Users/jasonmiller/Downloads/Process100/'
CLASS_DIRS=['0','1','2','3','4','5']
NUM_CLASSES=len(CLASS_DIRS)
IMG_FN="Process100_Image.csv"
RBC_FN="Process100_MergeRBC.csv"
NUC_FN="Process100_Nucleus.csv"
CEL_FN="Process100_ExpandCells.csv"
TIS_FN="Process100_Tissue.csv"
# Saving our models
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.01/'

In [4]:
# show columns
df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
initial_cols = df.columns
for col in initial_cols:
    print(col)

ImageNumber
ObjectNumber
FileName_Tumor
PathName_Tumor
AreaShape_Area
AreaShape_BoundingBoxArea
AreaShape_BoundingBoxMaximum_X
AreaShape_BoundingBoxMaximum_Y
AreaShape_BoundingBoxMinimum_X
AreaShape_BoundingBoxMinimum_Y
AreaShape_Center_X
AreaShape_Center_Y
AreaShape_CentralMoment_0_0
AreaShape_CentralMoment_0_1
AreaShape_CentralMoment_0_2
AreaShape_CentralMoment_0_3
AreaShape_CentralMoment_1_0
AreaShape_CentralMoment_1_1
AreaShape_CentralMoment_1_2
AreaShape_CentralMoment_1_3
AreaShape_CentralMoment_2_0
AreaShape_CentralMoment_2_1
AreaShape_CentralMoment_2_2
AreaShape_CentralMoment_2_3
AreaShape_Compactness
AreaShape_ConvexArea
AreaShape_Eccentricity
AreaShape_EquivalentDiameter
AreaShape_EulerNumber
AreaShape_Extent
AreaShape_FormFactor
AreaShape_HuMoment_0
AreaShape_HuMoment_1
AreaShape_HuMoment_2
AreaShape_HuMoment_3
AreaShape_HuMoment_4
AreaShape_HuMoment_5
AreaShape_HuMoment_6
AreaShape_InertiaTensorEigenvalues_0
AreaShape_InertiaTensorEigenvalues_1
AreaShape_InertiaTensor_0_0
Ar

In [5]:
df = None
GOOD_COLS=['ImageNumber']
GOOD_COLS.extend(['AreaShape_Area','AreaShape_Compactness','AreaShape_ConvexArea'])
GOOD_COLS.extend(['AreaShape_Eccentricity','AreaShape_EquivalentDiameter','AreaShape_EulerNumber'])
GOOD_COLS.extend(['AreaShape_Extent','AreaShape_FormFactor','AreaShape_MajorAxisLength'])
GOOD_COLS.extend(['AreaShape_MaxFeretDiameter','AreaShape_MaximumRadius','AreaShape_MeanRadius'])
GOOD_COLS.extend(['AreaShape_MinFeretDiameter','AreaShape_MinorAxisLength','AreaShape_Orientation'])
GOOD_COLS.extend(['AreaShape_Perimeter','AreaShape_Solidity'])
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_CentralMoment') ) ] )
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_NormalizedMoment') ) ] )
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_SpacialMoment') ) ] )  # yes
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_Zernike') ) ] )   # yes
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Granularity') ) ] )   # yes!!!
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Intensity') ) ] )   # yes!!!
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Neighbors') ) ] )   
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('RadialDistribution') ) ] )   
GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_') ) ] )  # yes
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Contrast_Hematoxylin') ) ] )   
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_DifferenceEntropy_Hematoxylin') ) ] ) 
#GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Entropy_Hematoxylin') ) ] )  # yes
GOOD_COLS

['ImageNumber',
 'AreaShape_Area',
 'AreaShape_Compactness',
 'AreaShape_ConvexArea',
 'AreaShape_Eccentricity',
 'AreaShape_EquivalentDiameter',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Texture_AngularSecondMoment_Eosin_3_00_256',
 'Texture_AngularSecondMoment_Eosin_3_01_256',
 'Texture_AngularSecondMoment_Eosin_3_02_256',
 'Texture_AngularSecondMoment_Eosin_3_03_256',
 'Texture_AngularSecondMoment_Eosin_4_00_256',
 'Texture_AngularSecondMoment_Eosin_4_01_256',
 'Texture_AngularSecondMoment_Eosin_4_02_256',
 'Texture_AngularSecondMoment_Eosin_4_03_256',
 'Texture_AngularSecondMoment_Eosin_5_00_256',
 'Texture_AngularSecondMoment_Eosin_5_01_256',
 'Texture_AngularSecondMoment_Eosin_5_02_256',
 'Texture_Angular

In [6]:
def nuc_prefix(col_name):
    return 'Nuc_'+col_name
def load_one_class(class_index):
    class_name=CLASS_DIRS[class_index]+'/'
    object_df = pd.read_csv(BASE_DIR+class_name+NUC_FN)
    print("Load %d nuclei, class %s." % (len(object_df),class_name))
    object_df = object_df[GOOD_COLS]
    object_df.rename(nuc_prefix,axis='columns',inplace=True)
    describe=False
    if describe:
        patch_df = object_df.groupby(['Nuc_ImageNumber']).describe() ## this is slow
        # The random forest classifier cannot handle two-part column names, so convert them to string.    
        # patch_df.columns=patch_df.columns.map(str)
        #patch_df.columns=patch_df.columns.map('_'.join)
    else:
        patch_df = object_df.groupby(['Nuc_ImageNumber']).max()
    patch_df.fillna(0,inplace=True)
    print("Load %d patches having nuclei." % len(patch_df))
    return patch_df
df = load_one_class(0)
df

Load 1826 nuclei, class 0/.
Load 100 patches having nuclei.


,Nuc_AreaShape_Area,Nuc_AreaShape_Compactness,Nuc_AreaShape_ConvexArea,Nuc_AreaShape_Eccentricity,Nuc_AreaShape_EquivalentDiameter,Nuc_AreaShape_EulerNumber,Nuc_AreaShape_Extent,Nuc_AreaShape_FormFactor,Nuc_AreaShape_MajorAxisLength,Nuc_AreaShape_MaxFeretDiameter,...,Nuc_Texture_Variance_Hematoxylin_4_02_256,Nuc_Texture_Variance_Hematoxylin_4_03_256,Nuc_Texture_Variance_Hematoxylin_5_00_256,Nuc_Texture_Variance_Hematoxylin_5_01_256,Nuc_Texture_Variance_Hematoxylin_5_02_256,Nuc_Texture_Variance_Hematoxylin_5_03_256,Nuc_Texture_Variance_Hematoxylin_7_00_256,Nuc_Texture_Variance_Hematoxylin_7_01_256,Nuc_Texture_Variance_Hematoxylin_7_02_256,Nuc_Texture_Variance_Hematoxylin_7_03_256
Nuc_ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,313,1.273323,345,0.564175,19.963065,1,0.626000,0.785346,22.632028,24.515301,...,5373.030388,5509.971988,5379.851952,5554.604152,5390.868264,5670.033081,5585.261228,5732.126976,5222.972048,6012.464844
2,336,2.208444,365,0.953539,20.683532,1,0.678977,0.762246,33.514234,32.572995,...,2427.560468,1795.349934,1962.047495,3130.013264,2745.314167,1729.433673,2007.641099,4345.990930,3406.000566,2028.824015
3,585,1.701673,603,0.945255,27.291851,1,0.785185,0.983658,30.773980,30.000000,...,4008.583910,4277.686559,3966.704980,4981.050625,4261.536108,4427.842961,3979.379198,4934.076397,4685.992476,4821.358674
4,635,2.727914,818,0.919852,28.434259,1,0.821256,0.990936,44.852725,43.416587,...,6102.481233,5980.225847,6252.637784,5521.856319,6391.851246,5899.618416,6492.630828,5798.265556,6203.178253,6126.136134
5,832,2.067581,896,0.952981,32.547432,1,0.814103,0.979789,49.273720,51.000000,...,3142.370723,3434.025250,3138.066261,3317.797463,3292.251918,3622.399506,3355.198244,4070.415978,4157.494126,4161.067762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,340,1.604151,356,0.937486,20.806284,1,0.702479,0.903434,32.719426,29.546573,...,3224.372980,3419.430862,3428.724138,3233.433129,3222.335047,3372.110056,3506.755562,3152.690753,3246.464268,2935.538854
97,812,2.097841,940,0.946143,32.153857,1,0.857143,0.916053,52.041988,49.929951,...,2333.062378,2625.735315,2508.707755,2401.475069,2388.904646,2844.753314,2335.631333,2563.962644,2375.814449,2535.747320
98,633,2.326529,702,0.968304,28.389446,1,0.810606,0.976125,47.357716,46.097722,...,2109.144977,2273.125654,2258.163534,2365.657747,2163.863799,2339.629347,2222.390678,2225.568501,2291.757707,2578.575725


In [7]:
print(datetime.datetime.now())
df = None
def load_all_classes():
    X = load_one_class(0)
    size = len(X)
    y = np.zeros(size)
    for i in range(1,NUM_CLASSES):
        Xi = load_one_class(i)
        size = len(Xi)
        yi = np.ones(size) * i
        X = pd.concat( (X,Xi) )
        y = np.concatenate( (y,yi) )
    return X,y
X,y=load_all_classes()
X

2022-05-26 09:24:50.924763
Load 1826 nuclei, class 0/.
Load 100 patches having nuclei.
Load 2435 nuclei, class 1/.
Load 100 patches having nuclei.
Load 886 nuclei, class 2/.
Load 100 patches having nuclei.
Load 1459 nuclei, class 3/.
Load 100 patches having nuclei.
Load 1713 nuclei, class 4/.
Load 100 patches having nuclei.
Load 3659 nuclei, class 5/.
Load 100 patches having nuclei.


,Nuc_AreaShape_Area,Nuc_AreaShape_Compactness,Nuc_AreaShape_ConvexArea,Nuc_AreaShape_Eccentricity,Nuc_AreaShape_EquivalentDiameter,Nuc_AreaShape_EulerNumber,Nuc_AreaShape_Extent,Nuc_AreaShape_FormFactor,Nuc_AreaShape_MajorAxisLength,Nuc_AreaShape_MaxFeretDiameter,...,Nuc_Texture_Variance_Hematoxylin_4_02_256,Nuc_Texture_Variance_Hematoxylin_4_03_256,Nuc_Texture_Variance_Hematoxylin_5_00_256,Nuc_Texture_Variance_Hematoxylin_5_01_256,Nuc_Texture_Variance_Hematoxylin_5_02_256,Nuc_Texture_Variance_Hematoxylin_5_03_256,Nuc_Texture_Variance_Hematoxylin_7_00_256,Nuc_Texture_Variance_Hematoxylin_7_01_256,Nuc_Texture_Variance_Hematoxylin_7_02_256,Nuc_Texture_Variance_Hematoxylin_7_03_256
Nuc_ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,313,1.273323,345,0.564175,19.963065,1,0.626000,0.785346,22.632028,24.515301,...,5373.030388,5509.971988,5379.851952,5554.604152,5390.868264,5670.033081,5585.261228,5732.126976,5222.972048,6012.464844
2,336,2.208444,365,0.953539,20.683532,1,0.678977,0.762246,33.514234,32.572995,...,2427.560468,1795.349934,1962.047495,3130.013264,2745.314167,1729.433673,2007.641099,4345.990930,3406.000566,2028.824015
3,585,1.701673,603,0.945255,27.291851,1,0.785185,0.983658,30.773980,30.000000,...,4008.583910,4277.686559,3966.704980,4981.050625,4261.536108,4427.842961,3979.379198,4934.076397,4685.992476,4821.358674
4,635,2.727914,818,0.919852,28.434259,1,0.821256,0.990936,44.852725,43.416587,...,6102.481233,5980.225847,6252.637784,5521.856319,6391.851246,5899.618416,6492.630828,5798.265556,6203.178253,6126.136134
5,832,2.067581,896,0.952981,32.547432,1,0.814103,0.979789,49.273720,51.000000,...,3142.370723,3434.025250,3138.066261,3317.797463,3292.251918,3622.399506,3355.198244,4070.415978,4157.494126,4161.067762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1323,3.283374,1474,0.925216,41.042611,1,0.837093,0.950693,64.861527,65.764732,...,2387.923601,2489.122089,2451.978878,2547.030549,2436.345727,2531.744140,2577.828444,2694.817106,2499.384383,2537.304120
97,923,2.000305,1052,0.943625,34.281192,1,0.610022,0.662628,58.363145,58.008620,...,2380.296123,2519.235596,2611.651575,2735.162840,2438.862034,2533.605642,2749.656414,2538.130546,2570.088740,2563.868007
98,1565,2.337659,1833,0.956790,44.638771,1,0.824017,0.971563,68.549994,70.213959,...,1191.651759,1317.199289,1193.370674,1438.788386,1134.684773,1323.817397,1220.944704,1412.783053,1144.578125,1462.124728


In [8]:
print(datetime.datetime.now())
Xtrain,Xvalid,ytrain,yvalid = train_test_split(X, y.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xtrain.shape,'ytrain',ytrain.shape,'non-zero:',np.count_nonzero(ytrain))
print('Xvalid',Xvalid.shape,'yvalid',yvalid.shape,'non-zero:',np.count_nonzero(yvalid))

2022-05-26 09:24:53.060580
Xtrain (450, 433) ytrain (450,) non-zero: 375
Xvalid (150, 433) yvalid (150,) non-zero: 125


In [9]:
# RandomForestClassifier can only track feature names of type string.
num_problems=0
for name in Xtrain.columns:
    if not isinstance(name,str):
        num_problems += 1
        print(type(name),name)
if num_problems==0:
    print("Ok")

Ok


In [10]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model.fit(self.Xtr,self.ytr)
        #print(dir(self.model))  # see whether feature_names_in_ got created
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  # bug fix
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

2022-05-26 09:24:53.161401


In [11]:
print('Train on all Features')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
rf1.set_validation(Xvalid,yvalid)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())
print('The impurity-based feature importances.')
top = rf1.important_features()
top.head()

Train on all Features
2022-05-26 09:24:53.184966
2022-05-26 09:24:53.705083
Accuracy: 85.33333333333333
Confusion:
[[22  0  1  2  0  0]
 [ 0 23  0  1  0  2]
 [ 1  0 16  4  0  1]
 [ 0  1  3 14  0  0]
 [ 0  0  0  0 24  1]
 [ 0  2  1  1  1 29]]
The impurity-based feature importances.


,0,1
0,0.026285,Nuc_Texture_SumAverage_Eosin_4_03_256
1,0.024400,Nuc_Texture_SumAverage_Eosin_5_03_256
2,0.019538,Nuc_Texture_SumAverage_Eosin_5_02_256
3,0.018081,Nuc_Texture_SumAverage_Eosin_5_01_256
4,0.016343,Nuc_Texture_SumAverage_Eosin_3_03_256


GPU was not active:  
2022-05-25 16:35:28.165143  
2022-05-25 16:35:28.524197

GPU probably not active:  
2022-05-25 16:50:42.142147  
2022-05-25 16:50:42.486454

Why isn't pandas using the GPU?  
Why isn't RF using the GPU?